In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import pipeline
import torch
import gradio as gr

## **Load GPT-2 for Generation**

In [ ]:
print("🔁 Loading GPT-2 model...")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_model.config.pad_token_id = gpt2_tokenizer.eos_token_id

def generate_fake_news(prompt="Breaking News:"):
    if not prompt.strip():
        return "⚠️ No topic provided for generation."

    input_ids = gpt2_tokenizer.encode(prompt, return_tensors='pt')
    attention_mask = torch.ones_like(input_ids)

    outputs = gpt2_model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=30,
        num_return_sequences=5,
        do_sample=True,
        temperature=0.9,
        top_k=50,
        top_p=0.95,
        pad_token_id=gpt2_tokenizer.eos_token_id
    )

    headlines = []
    for output in outputs:
        headline = gpt2_tokenizer.decode(output, skip_special_tokens=True)
        headlines.append(headline)
    return "\n".join([f"{i+1}. {h}" for i, h in enumerate(headlines)])

🔁 Loading GPT-2 model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## **Load BERT for Detection**

In [ ]:
print("🔁 Loading BERT classifier pipeline...")
clf = pipeline(
    "text-classification",
    model="jy46604790/Fake-News-Bert-Detect",
    tokenizer="jy46604790/Fake-News-Bert-Detect"
)

def detect_fake_news(text):
    if not text.strip():
        return "⚠️ No news text provided for detection."

    result = clf(text[:500])[0]  # truncate long texts
    label = "🟥 FAKE" if result['label'] == "LABEL_0" else "🟩 REAL"
    confidence = round(result['score'] * 100, 2)
    return f"📝 Input: {text}\n\n🔍 Prediction: {label}\n✅ Confidence: {confidence}%"

🔁 Loading BERT classifier pipeline...


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu


## **Gradio Interface Function**

In [ ]:
def interface(prompt, text):
    fake_news = generate_fake_news(prompt) if prompt.strip() else "⚠️ No topic entered."
    detection = detect_fake_news(text) if text.strip() else "⚠️ No news text entered."
    return fake_news, detection

## **Gradio App**

In [ ]:
demo = gr.Interface(
    fn=interface,
    inputs=[
        gr.Textbox(label="🧠 Enter a Topic for Fake News", placeholder="e.g. Budget 2025, Alien Invasion"),
        gr.Textbox(label="📄 Paste News to Detect", placeholder="e.g. Government announces free petrol every Monday...")
    ],
    outputs=[
        gr.Textbox(label="📰 Generated Fake News"),
        gr.Textbox(label="🕵️‍♂️ Detection Result")
    ],
    title=" Fake News Generator & Detector using Generative AI and NLP",
    description="\n",
    allow_flagging="never"
)

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


In [ ]:
demo.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f21f7368c0569c7ee7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
